In [2]:
import yaml
import pickle as pkl
import json
from tqdm import tqdm 
from collections import Counter
import os

In [3]:
with open("../../config.yaml", "r") as ymlfile:
    CFG = yaml.load(ymlfile, Loader=yaml.SafeLoader)

In [7]:
base_file="bp_3jobs_desc_edu_skills_industry_date_company_FR.json"
datadir = CFG["gpudatadir"]
input_file= os.path.join(datadir, base_file)


In [11]:
with open(os.path.join(CFG["gpudatadir"], "ind_class_dict.pkl"), 'rb') as f:
    ind_dict = pkl.load(f)
with open(os.path.join(CFG["gpudatadir"], "good_skills.p"), 'rb') as f:
    skills = pkl.load(f)
rev_ind_dict = {v:k for k,v in ind_dict.items()}

In [12]:
with open(input_file, 'r') as f:
    num_lines = sum(1 for line in f)

In [13]:
ind_counter = Counter()

In [15]:
with open(input_file, 'r') as f:
    pbar = tqdm(f, total=num_lines)
    for line in f:
        raw_p = json.loads(line)
        ind_counter[raw_p[4]] +=1
        pbar.update(1)

100%|██████████| 850902/850902 [00:50<00:00, 25072.54it/s]

In [18]:
most_common_ind = [i[0] for i in ind_counter.most_common(15)]
print(most_common_ind)

['Marketing and Advertising', 'Information Technology and Services', '', 'Human Resources', 'Internet', 'Banking', 'Computer Software', 'Research', 'Mechanical or Industrial Engineering', 'International Trade and Development', 'Retail', 'Government Administration', 'Management Consulting', 'Public Relations and Communications', 'Food Production']


In [19]:
skills

['Graphic Design',
 'Web Design',
 'Réseaux sociaux',
 'Marketing',
 'Microsoft Excel',
 'PowerPoint',
 'Microsoft Word',
 'Microsoft Office',
 'Anglais',
 'Stratégie marketing',
 'Communication marketing',
 'Service client',
 'Gestion de projet',
 'Management',
 'Retail',
 'Negotiation',
 'Customer Service',
 'Forecasting',
 'Business Strategy',
 'Merchandising',
 'Sales',
 'Team Management',
 'Purchasing',
 'Change Management',
 'PL/SQL',
 'XML',
 'Agile Methodologies',
 'Microsoft SQL Server',
 'Software Project Management',
 'Méthodes agiles',
 "Gestion d'équipe",
 'Stratégie IT',
 'Gestion de projet logiciel',
 'Scrum',
 'Architecture',
 'UML',
 '.NET',
 'Intégration',
 'Consulting',
 'SQL',
 'Oracle',
 'Data Analysis',
 'Teamwork',
 'Français',
 'Communication',
 "Travail d'équipe",
 'Recherche',
 'English',
 'Marketing Communications',
 'Java',
 'Python',
 'C++',
 'C',
 'LaTeX',
 'R',
 'Operations Management',
 'Contract Negotiation',
 'Contract Management',
 'Négociation de con

In [20]:
def handle_skills(skill_list, good_skills):
    skills_ind = []
    for sk in skill_list:
        if sk in good_skills:
            skills_ind.append(sk)
    return skills_ind

In [24]:
tuples = []
counter = 0
with open(input_file, 'r') as f:
    pbar = tqdm(f, total=num_lines, desc="Building toy dataset")
    for line in f:
        raw_p = json.loads(line)
        if raw_p[4] in most_common_ind and counter < 5000:
            tuples.append({
                "id": raw_p[0],
                "jobs": raw_p[1],
                "skills": handle_skills(raw_p[2], skills),
                "edu": raw_p[3],
                "industry": raw_p[4]
            })
            counter += 1
        pbar.update(1)


100%|██████████| 850902/850902 [13:26<00:00, 1054.80it/s] 00:55, 15150.03it/s]

Building toy dataset:  27%|██▋       | 228697/850902 [00:09<00:25, 24179.09it/s]


Building toy dataset:  55%|█████▍    | 464041/850902 [00:19<00:15, 24220.62it/s]


Building toy dataset:  82%|████████▏ | 698788/850902 [00:29<00:06, 23944.42it/s]


Building toy dataset: 100%|██████████| 850902/850902 [00:53<00:00, 24186.29it/s]

In [26]:
len(tuples)

5000

In [27]:
with open(os.path.join(CFG["gpudatadir"], "CV_5000_PLDAC.pkl"), 'wb') as f:
    pkl.dump(tuples, f)